# PyTorch

파이토치(PyTorch)는 딥러닝 연구와 개발에서 널리 사용되는 오픈소스 딥러닝 프레임워크입니다.
- 기복적으로 `NumPy`와 유사한 다차원 배열 연산을 지원하며, 추가적으로 `GPU`를 이용한 연산과 자동 미분(`Autograd`) 기능을 제공합니다.
- 또한 데이터 전처리, 모델 정의, 최적화(Optimizer), 손실 함수(Loss Function) 등 딥러닝 구현에 필요한 다양한 기능들을 직관적이고 편리하게 제공합니다

## 텐서(Tensors)
- 텐서(Tensor)는 파이토치의 가장 기본적인 데이터 구조로, `NumPy`의 `ndarray`와 매우 유사하게 다차원 배열 데이터를 다룰 수 있습니다.
- 파이토치 `Tensor`가 NumPy 배열과 구별되는 가장 큰 특징은 **GPU**와 같은 특수한 하드웨어를 활용해 연산 속도를 획기적으로 향상시킬 수 있다는 점입니다.

In [ ]:
import torch

In [ ]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)

print(x)
print(type(x))
print(f"\nShape of tensor: {x.shape}")
print(f"Datatype of tensor: {x.dtype}")

### GPU 연산
- `torch.Tensor`는 ``.to`` 메서드를 이용해 CPU → GPU 또는 GPU → CPU 등 다른 디바이스로 옮길 수 있습니다.
- GPU 위에 있는 텐서끼리는 연산을 할 수 있습니다.

In [ ]:
x1 = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)
x2 = torch.ones((3, 2), dtype=torch.float32)

if torch.cuda.is_available():
    x1_gpu = x1.to("cuda")
    x2_gpu = x2.to("cuda")
    print(f"x1 tensor is stored on device: {x1.device}")
    print(f"x1_gpu tensor is stored on device: {x1_gpu.device}")

    y = torch.matmul(x1_gpu, x2_gpu) # Matrix multiplication on GPU
    print("\nx1_gpu @ x2_gpu = \n", y)

## Autograd: Automatic Differentiation (자동 미분)

PyTorch에서 딥러닝의 핵심이 되는 기능 중 하나는 `autograd` 패키지입니다.

``autograd``는 Tensor의 모든 연산에 대응하는 미분(gradient)을 자동으로 계산해주는 기능입니다.

1. ``torch.Tensor``의 ``.requires_grad`` 속성을 ``True``로 설정하게 되면 ``autograd``패키지는 이 텐서에 수행되는 모든 연산을 자동으로 추적합니다.
2. 연산이 끝난뒤 ``.backward()`` 메서드를 호출하면, 연산 그래프를 따라 모든 gradient가 자동으로 계산됩니다.
3. 계산된 gradient는 각 텐서의 ``.grad`` 속성에 저장됩니다

함수 $f = x^2 + y^2 + z^2$를 예로 들어 살펴보겠습니다.

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)
z = torch.tensor(1.5, requires_grad=True)
f = x**2+y**2+z**2

print(f"grads before backward(): ", x.grad, y.grad, z.grad)
f.backward()
print(f"grads after backward():  ", x.grad, y.grad, z.grad)

`backward()`함수는 모든 leaf node `x`, `y`, `z`에 대해 미분을 계산하여 `.grad`값에 저장합니다.
$$\frac{\partial f}{\partial x} = 2x = 4, \quad \frac{\partial f}{\partial y} = 2y = 6, \quad \frac{\partial f}{\partial z} = 2z = 3$$

# Image Classification using CNNs

이미지 분류 모델은 아래 과정을 통해 학습합니다.

1. ``torchvision``을 이용하여 이미지 데이터셋을 읽어온다.
2. 신경망(Neural Network)과 학습 가능한 파라미터(learnable parameter)들을 정의한다.
3. loss 함수를 정의한다.
4. 학습(training): 학습데이터를 순회하며 순전파(forward propagation)와 역전파(backward propagation)을 수행하고 경사하강법(gradient descent)에 따라 파라미터를 업데이트한다.
5. 평가(evaluation): validation/test 데이터를 이용하여 모델을 평가한다.

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm

from helpers import visualize_few_samples


## 1. Dataset & DataLoader
PyTorch는 데이터를 효율적으로 관리하고 학습에 활용할 수 있도록 아래 두 가지 핵심 모듈을 제공합니다:
1. ``torch.utils.data.Dataset``: 한번에 하나의 데이터와 라벨을 읽어옴
2. ``torch.utils.data.DataLoader``: `Dataset` 객체를 받아 mini-batch로 데이터를 묶어주며, shuffle, 병렬 처리 등을 지원함.

이번 실습에서는 CIFAR-10 dataset을 이용하여 이미지 분류 모델을 학습해봅니다.

CIFAR-10은 컴퓨터 비전 분야에서 널리 사용되는 벤치마크 이미지 데이터셋으로,
- 각 이미지는 32×32 해상도의 3채널(RGB) 컬러 이미지입니다.
- 비행기(airplane), 자동차(automobile), 새(bird), 고양이(cat) 등 총 10개의 클래스로 구성되어 있습니다.


In [ ]:
def load_cifar10_datasets(data_root_dir):
    transform = transforms.Compose([
        transforms.ToTensor(), # convert PIL Image to a torch.Tenso of shape (C, H, W) and in the range [0.0, 1.0].
        transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)) 
    ])
    
    train_dataset = datasets.CIFAR10(root=data_root_dir, train=True, download=False, transform=transform)
    test_dataset = datasets.CIFAR10(root=data_root_dir, train=False, download=False, transform=transform)

    return train_dataset, test_dataset

PyTorch의 `Dataset` 객체는 **시퀀스(Sequence) 컨테이너처럼 동작**합니다.  

- `dataset[i]` : 인덱스 `i`에 해당하는 샘플(example)을 반환합니다. 일반적으로 `(image_tensor, label)` 형태의 튜플입니다.
- `len(dataset)` : 전체 데이터 개수를 반환합니다.


In [ ]:
train_dataset, test_dataset = load_cifar10_datasets("/datasets")

print("train_dataset size: ", len(train_dataset))
print("test_dataset size: ", len(test_dataset))
print("train_dataset[0] image shape: ", train_dataset[0][0].shape) # RGB image of shape (3, 32, 32)
print("train_dataset[0] label: ", train_dataset[0][1])  # label is an integer

In [ ]:
visualize_few_samples(train_dataset, cols = 10, rows = 2)

`DataLoader`는 `Dataset`객체를 기반으로 데이터를 효율적으로 불러오는 도구입니다.

- `Dataset`에서 샘플(example)들을 가져와 mini-batch 단위로 묶어주는
- `shuffle = True`를 통해 매 epoch마다 데이터 순서를 랜덤하게 섞어줌
- `num_workers`를 통해 데이터 전처리를 multiprocessing으로 수행할 수 있음

In [ ]:
batch_size = 64

train_dataloader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_dataloader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
    
for X, y in test_dataloader:    # DataLoader is an iterable
    print(f"Shape of X: {X.shape}")     # shape: [batch_size, C, H, W]
    print(f"Shape of y: {y.shape}, dtype: {y.dtype}")
    break

## 2. Network 정의하기
딥러닝 모델 $f$는 **RGB 이미지** $\mathbf{x}$를 입력받아, 각 클래스에 대한 점수(logit) $\mathbf{z}$를 출력하는 함수입니다

$$
f : \mathbf{x} \;\;\mapsto\;\; \mathbf{z}
$$

- 입력: $\mathbf{x} \in \mathbb{R}^{3 \times 32 \times 32}$  (채널 3개, 높이 32, 너비 32인 컬러 이미지)  
- 출력: $\mathbf{z} \in \mathbb{R}^{10}$  (10개 클래스에 대한 logit 벡터)  

즉, 모델 $f$는 다음과 같은 함수로 표현됩니다:  

$$
f : \mathbb{R}^{3 \times 32 \times 32} \;\;\to\;\; \mathbb{R}^{10}
$$

### Logit과 확률 분포  

딥러닝 모델을 통해 예측된 logit값 $\mathbf{z}$는 구간 $(-\infty, \infty)$에 분포하는 실수 벡터이며, 이 값은 **Softmax 함수**를 통해 $[0, 1]$사이의 확률값을 가지는 확률 벡터로 변환됩니다. 

$$
\hat{\mathbf{y}} = \text{Softmax}(\mathbf{z}), 
\quad \hat{\mathbf{y}} = (\hat{y}_1, \hat{y}_2, \dots, \hat{y}_{10}) \in [0,1]^{10}, 
\quad \sum_{k=1}^{10} \hat{y}_k = 1
$$  

$\hat{\mathbf{y}}$의 각 원소 $\hat{y}_c$는 입력 이미지 $\mathbf{x}$가 클래스 $c$에 속할 확률을 의미합니다.  

- 주의: PyTorch에서는 `nn.CrossEntropyLoss()`가 내부적으로 Softmax 연산을 포함하고 있기 때문에, 일반적으로 `forward()` 함수에서는 logit값 $\mathbf{z}$를 반환하며, Softmax는 별도로 적용하지 않습니다.  

---

<mark>실습</mark> 아래 구조를 가지는 `SimpleCNN`을 완성하세요
1. **Conv1**: Convolutional layer (5x5 kernel size, 8 filters, strides of 1, no zero padding) + ReLU activation
2. **Pool1**: Max pooling layer (2x2 kernel size, strides 2)
3. **Conv2**: Convolutional layer (5x5 kernel size, 16 filters, strides of 1, no zero padding) + ReLU activation
4. **Pool2**: Max pooling layer (2x2 kernel size, strides 2)
5. **FC1**: Fully connected layer with 128 output units + ReLU activation
6. **FC2**: Fully connected layer with `out_dim` output units (`out_dim`: 클래스 수)

힌트
- CIFAR-10 데이터셋 입력 이미지의 shape은 `(3, 32, 32)` 입니다
- [`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html): 순서가 있는 모듈들의 컨테이너(container)로, 데이터는 컨테이너 내의 모든 모듈들을 정의된것과 같은 순서로 통과합니다.
- [`nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
- [`nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)
- [`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) 혹은 [`torch.tensor.view`](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html)를 활용하여 3차원 텐서를 1차원 텐서로 펼쳐줍니다.
- [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear): 입력 텐서에 대한 선형 변환 $y=Wx+b$를 수행합니다 (Fully Connected Layer)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, out_dim):
        super().__init__()
        ##### YOUR CODE START #####



        ##### YOUR CODE END #####

    def forward(self, x):
        ##### YOUR CODE START #####



        ##### YOUR CODE END #####

        return logits

In [ ]:
# Test forward pass
model = SimpleCNN(out_dim = 10)

X = torch.rand(16, 3, 32, 32) # dummy data for testing with batch_size 16
logits = model(X) 

print("logits.shape: ", logits.shape)

<mark>실습</mark> 앞서 정의한 `model = SimpleCNN(out_dim=10)`의 <b>학습 가능한 파라미터 수(learnable parameters)</b>를 직접 손으로 계산해 보세요.  
- 각 레이어의 **가중치(weight)**와 **편향(bias)**을 모두 포함해야 합니다.  
- 결과는 숫자 값으로 기입하거나, 숫자 계산식(예: 5*10)으로 입력해도 괜찮으나 **파이썬 변수를 사용하지 마세요**.

In [ ]:
num_params_conv1 = ...  # TODO: number of parameters in Conv1 layer
num_params_pool1 = ...  # TODO: number of parameters in Pool1 layer
num_params_conv2 = ...  # TODO: number of parameters in Conv2 layer
num_params_pool2 = ...  # TODO: number of parameters in Pool2 layer
num_params_fc1 = ...  # TODO: number of parameters in FC1 layer
num_params_fc2 = ...  # TODO: number of parameters in FC3 layer

In [ ]:
total_params = (num_params_conv1  + num_params_pool1 + num_params_conv2 + num_params_pool2 + num_params_fc1 + num_params_fc2)

print(f"Total number of params : {total_params}")

assert sum(p.numel() for p in model.parameters() if p.requires_grad) == total_params, "❌ 계산한 파라미터 수가 실제 모델과 일치하지 않습니다."
print('\033[92mAll tests passed!')

## 3. Forward and Backward Propagation

정의된 딥러닝 모델과 데이터셋을 이용하여 학습을 진행하려면 먼저 <b>손실 함수(Loss function)</b>를 정의해야 합니다.  
분류 문제에서는 **Categorical Cross Entropy Loss**가 일반적으로 사용되며, PyTorch에서는 `nn.CrossEntropyLoss()`를 통해 제공됩니다. 

### 순전파 (Forward Pass)
1. 입력 데이터 `X`를 `model`에 전달하여 출력값 `logits`을 계산합니다.  
2. 모델 출력값 `logits`과 타겟 레이블 `y`를 손실 함수에 전달하여 `loss` 값을 계산합니다.  

### 역전파 (Backward Pass)
1. 계산된 `loss` 값에 `.backward()` 메서드를 호출하면, 모델 내부의 모든 **학습 가능한 파라미터**에 대해 손실 함수의 미분(gradient)이 자동으로 계산됩니다.  
2. 이렇게 얻어진 gradient는 추후 gradient descent <b>최적화(optimizer)</b>알고리즘에서 파라미터를 업데이트할 때 사용됩니다.  


In [ ]:
batch_size = 16
criterion = nn.CrossEntropyLoss()

model = SimpleCNN(out_dim = 10)
X = torch.rand(batch_size, 3, 32, 32)   # dummy image data with shape (batch_size, 1, 28, 28)
y = torch.randint(10, (batch_size,))    # dummy target labels

logits = model(X)   # forward pass
loss = criterion(logits, y)

print("----- Model Parameters before backward() -----")
for name, param in model.named_parameters():
    print(f"Layer {name}\t| Parameter shape: {param.shape}\t| grad : {param.grad}")

loss.backward()   # backward pass

print("\n----- Model Parameters after backward() -----")
for name, param in model.named_parameters():
    print(f"Layer {name}\t| Parameter shape: {param.shape}\t| grad : {param.grad.shape}")

## 4. Optimizing your model parameters

역전파를 통해 계산된 gradient값을 이용하여 모델의 파라미터를 업데이트하는 과정을 최적화(optimization)이라고 합니다.

- 딥러닝에서는 주로 mini-batch Stochastic Gradient Descent (SGD)알고리즘이 사용되며, PyTorch에서는 `torch.optim.SGD`를 통해 제공됩니다.
- `torch.optim`에서 제공하는 `optimizer` 객체는 학습 대상이 되는 모델의 파라미터를 전달받아 파라미터 업데이트를 자동으로 수행해줍니다.
- 사용자는 `optimizer.step()`만 호출하면 복잡한 업데이트 로직을 직접 작성할 필요 없이 `optimizer`가 대신 가중치를 갱신해줍니다.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.01) # Optimizer 초기화시 모델 파라미터들을 등록(register)

print("First few elements of Model params (before optimizer step):")
print(next(model.parameters()).view(-1)[:10])

# A single training step consist of:
optimizer.zero_grad()             # 1. 이전 스텝이서 계산된 gradient를 초기화
logits = model(X)                 # 2. Forward pass
loss = criterion(logits, y)       # 3. Loss 계산
loss.backward()                   # 4. Backward pass
optimizer.step()                  # 5. Optimerzer가 Parameter update를 대신 해준다.

print("\nFirst few elements of Model params (after optimizer step):")
print(next(model.parameters()).view(-1)[:10])

### 딥러닝 학습의 기본 흐름

for each iteration (called an **epoch**):
- forward pass: 입력 데이터를 통해 모델의 예측값 (output) 계산
- loss 계산: 예측값과 정답(target)의 차이를 계산해 error(loss)를 측정
- Backward pass: 파라미터에 대한 loss의 미분 계산
- Optimization: 경사하강법(gradient descent)를 통하여 파라미터 업데이트

각 epoch은 다음의 과정으로 구성됩니다.
- Train Loop: 학습 데이터를 이용하여 모델이 최적의 파라미터를 학습
- Evaluation(Validation/Test) Loop - 검증/테스트 데이터셋을 이용하여 모델의 성능을 평가.

<mark>실습</mark> `train_one_epoch`를 완성하세요

In [ ]:
def train_one_epoch(model, device, dataloader, criterion, optimizer, epoch):
    """ train for one epoch """

    model.train() # switch to train mode
    total_loss = 0.0

    dataloader_tqdm = tqdm(dataloader, desc=f'Training Epoch {epoch + 1}', total=len(dataloader))
    
    for X, y in dataloader_tqdm:
        X = X.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        ##### YOUR CODE START #####
        # Forward propagation and compute loss


        # Backpropagation and update parameter


        ##### YOUR CODE END #####

        total_loss += loss.item()
        
        dataloader_tqdm.set_postfix({"loss": f"{loss.item():.4e}"})

    dataloader_tqdm.close()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
batch_size = 64
train_dataset, test_dataset = load_cifar10_datasets("/datasets")
train_dataset_subset = torch.utils.data.Subset(train_dataset, range(0, 100))
train_dataloader = DataLoader(dataset= train_dataset_subset, batch_size=batch_size, shuffle=True)

model = SimpleCNN(out_dim = 10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

avg_loss = train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch=0)

<mark>실습</mark> `evaluate_one_epoch`를 완성하세요

1. Forward propagation
   - 딥러닝 모델 $f$에 입력 `X`를 전달하여 `logits`값 $\mathbf{z} \in \mathbb{R}^C$를 계산합니다. ($C$: 클래스 개수)
    $$ f : \mathbb{R}^{3 \times 32 \times 32} \;\;\to\;\; \mathbb{R}^{C}, \quad \mathbf{z} = f(\mathbf{X}) \in \mathbb{R}^C$$

2. Softmax (optional)
   - `logits`값을 `Softmax` 함수에 통과시키 각 클래스에 대한 **확률 분포**를 얻습니다.
    $$
    \hat{\mathbf{y}} =   
    \begin{bmatrix}  \hat{y}_{1} & \hat{y}_{2} & \cdots & \hat{y}_{C} \end{bmatrix}
    =  \mathrm{softmax}(\mathbf{z}) \in \mathbb{R}^{C}
    $$
   - 여기서 $\hat{y}_j$는 입력 $\mathbf{x}$가 클래스 $j$에 속할 확률을 예측한 값입니다

3. 예측 (Prediction)
   - 최종적으로 우리는 다음과 같이 **가장 확률이 높은 클래스**를 예측값으로 선택합니다:
    $$
    y_{pred} = \arg\max_j \hat{y}_j
    $$
   - $ y_{pred} \in \{0, 1, \dots, C-1\}$ 는 예측된 클래스 레이블을 나타냅니다.


4. 실제로는 이러한 작업이 **mini-batch 단위**로 처리됩니다:
    $$ \hat{\mathbf{Y}} \in \mathbb{R}^{m \times C} $$

    - $m$: 미니배치 크기

힌트: 아래 함수를 활용하세요
- [`nn.Softmax()`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
- [`torch.argmax()`](https://pytorch.org/docs/stable/generated/torch.argmax.html)
- [`torch.Tensor.item()`](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html)
- [`torch.Tensor.type()`](https://docs.pytorch.org/docs/stable/generated/torch.Tensor.type.html)
- [`torch.sum()`](https://docs.pytorch.org/docs/stable/generated/torch.sum.html)


In [ ]:
def evaluate_one_epoch(model, device, dataloader, criterion, epoch = 0):
    """ Evaluate the model for one epoch"""
    model.eval() # Set the model to evaluation mode. important for batch normalization and dropout layers

    total_loss, total_correct = 0.0, 0.0
    
    with torch.no_grad():   # disable gradient calculation
        dataloader_tqdm = tqdm(dataloader, desc='Validation/Test', total=len(dataloader))
        for X, y in dataloader_tqdm:
            X, y = X.to(device), y.to(device)
            
            ##### YOUR CODE START #####
            # Forward propagation and compute loss


            # calculate y_pred using argmax



            # accumulate total_loss (sum of loss over mini-batches). Use .item() to obtain the value of a tensor as a Python number
            # accumulate total_correct (the number of correct predictions). Use .item() to obtain the value of a tensor as a Python number



            ##### YOUR CODE END #####

            dataloader_tqdm.set_postfix({"loss": f"{loss.item():.4e}"})
        dataloader_tqdm.close()

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / len(dataloader.dataset)
    print(f"Test Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {avg_loss:>8f}\n")

    return(avg_loss, accuracy)

In [ ]:
test_dataset_subset = torch.utils.data.Subset(test_dataset, range(0, 100))
test_dataloader = DataLoader(dataset= test_dataset_subset, batch_size=batch_size, shuffle=False)

test_avg_loss, test_accuracy = evaluate_one_epoch(model, device, test_dataloader, criterion)

## 5. 학습 및 평가

<mark>실습</mark> CNN 이미지 분류 모델을 직접 학습(train)해보고 결과를 살펴보세요

In [ ]:
def main():
    # Hyperparameters
    batch_size = 128     # Use ~ 530MB of VRAM
    learning_rate = 1e-2
    num_epochs = 10

    # training setting
    num_workers = 4
    device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


    train_dataset, test_dataset = load_cifar10_datasets("/datasets")
    num_classes = len(train_dataset.classes)
    
    train_dataloader = DataLoader(dataset= train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_dataloader = DataLoader(dataset= test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    model = SimpleCNN(out_dim = num_classes).to(device)   # move model to device (GPU)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9, weight_decay=5e-4)

    print(f"Using {device} device")
    print(f"Number of model parameters: {sum(p.numel() for p in model.parameters())}")

    for epoch in range(num_epochs):
        train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch)
        evaluate_one_epoch(model, device, test_dataloader, criterion, epoch)

In [ ]:
main()

코드를 제대로 구현하셨다면 하이퍼파라미터 튜닝(hyperparameter tuning)을 따로 하지 않아도 `test accuracy > 60%`를 달성하실 수 있습니다

<mark>실습</mark> 더 높은 이미지 분류 정확도를 달성하기 위해서는 어떤 방법들이 효과적일지 고민해보세요.
- 네트워크 구조 변경 (더 깊거나 넓은 CNN)  
- Regularization 기법(배치 정규화, 드롭아웃 등)
- 데이터 증강(data augmentation)  
- 전이 학습(Transfer learning)
- 학습률(learning rate) 및 스케쥴러(scheduler)
- Other optimizers